In [47]:
!pip install tushare
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [48]:
import tushare as ts
from dotenv import load_dotenv
import pandas as pd 
from tqdm import tqdm
import os

In [49]:
load_dotenv()
token = os.getenv("API_TOKEN_TUSHARE")
ts.set_token(token)
pro = ts.pro_api()

In [50]:
all_data = pro.daily(exchange='', start_date='20210101', end_date='20240101')

In [ ]:
all_data

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,002276.SZ,20231229,10.04,10.22,9.99,10.22,10.02,0.20,1.9960,98384.38,99974.925
1,603583.SH,20231229,19.80,20.15,19.70,20.03,19.83,0.20,1.0086,36258.00,72553.459
2,300447.SZ,20231229,14.16,14.51,14.13,14.50,14.21,0.29,2.0408,27432.84,39478.857
3,601886.SH,20231229,6.45,6.68,6.45,6.66,6.51,0.15,2.3041,79960.71,52827.192
4,603682.SH,20231229,5.73,5.77,5.68,5.74,5.72,0.02,0.3497,69734.00,39897.424
...,...,...,...,...,...,...,...,...,...,...,...
5995,600250.SH,20231228,12.03,12.79,11.42,11.43,12.30,-0.87,-7.0732,492175.06,592963.643
5996,002858.SZ,20231228,16.29,17.99,15.90,17.99,16.35,1.64,10.0306,147490.66,250989.086
5997,603161.SH,20231228,15.51,15.84,15.36,15.76,15.57,0.19,1.2203,29292.00,45970.210
5998,002579.SZ,20231228,8.43,8.63,8.36,8.57,8.44,0.13,1.5403,81933.05,69889.038


In [ ]:
def query_suspension(stock_code):
    try:
        suspension_data = pro.suspend(ts_code=stock_code, start_date='20210101', end_date='20240101')
        return suspension_data
    except Exception as e:
        print(f"Error fetching data for {stock_code}: {e}")
        return pd.DataFrame() 

all_suspension = pd.DataFrame()
for stock in tqdm(all_data['ts_code'].unique()):
    suspension_results = query_suspension(stock)
    all_suspension = pd.concat([suspension_results, all_suspension])
all_suspension

  1%|▏         | 76/5328 [01:05<1:14:59,  1.17it/s]


KeyboardInterrupt: 

In [ ]:
all_data['trade_date'] = pd.to_datetime(all_data['trade_date'])
all_suspension['suspend_date'] = pd.to_datetime(all_suspension['suspend_date'])
all_suspension['resume_date'] = pd.to_datetime(all_suspension['resume_date'])
def is_not_suspended(row, suspension_df):
    ts_code = row['ts_code']
    trade_date = row['trade_date']
    suspensions = suspension_df[suspension_df['ts_code'] == ts_code]
    for _, suspension in suspensions.iterrows():
        if suspension['suspend_date'] <= trade_date <= suspension['resume_date']:
            return False  
    return True  
filtered_data = all_data[all_data.apply(is_not_suspended, axis=1, suspension_df=all_data)]

In [ ]:
filtered_data.to_csv('data/filtered_data.csv')